## Goal
Programmatically run PAMl analysis with different input alignment, tree and options, and collect the results for output. This helps with reproducibility and makes documenting the analysis easier.

## Approach
Use `Biopython`'s integration for PAML.

In [1]:
from Bio.Phylo.PAML import codeml
import os

In [23]:
# store the script directory
script_dir = os.path.abspath('')
print(script_dir)

/Users/bhe2/Documents/work/current/C037-Cand-auris-adhesin/02-case-studies/09-natural-selection/script


## Analysis

### p1-414 / branch one ratio / codonfreq = 1

In [39]:
# change to the first segment analysis dir
working_dir = "../output/paml/B8441-OG-part/p1-414/branch-1r-cf1/"
os.chdir(working_dir)

In [40]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [41]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1,
    model = 0,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [54]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [55]:
res = cml.run()

In [62]:
sum_res = {"branch-1r-cf1": res}

### p1-414 / branch free ratio / codonfreq = 1

In [68]:
# change to the first segment analysis dir
working_dir = "../branch-freer-cf1/"
os.chdir(working_dir)

In [69]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [74]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 1,
    clock = 0,
    model = 1,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [75]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [76]:
res = cml.run()

In [77]:
sum_res = {"branch-freer-cf1": res}

### p1-414 / branch one ratio / codonfreq = 2

In [83]:
# change to the first segment analysis dir
working_dir = "../branch-1r-cf2/"
os.chdir(working_dir)

In [40]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [84]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2,
    model = 0,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [85]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [86]:
res = cml.run()

In [87]:
sum_res = {"branch-1r-cf2": res}

### p1-414 / branch free ratio / codonfreq = 2

In [88]:
# change to the first segment analysis dir
working_dir = "../branch-freer-cf2/"
os.chdir(working_dir)

In [89]:
cml = codeml.Codeml()
cml.working_dir = "./"
cml.alignment = "../B8441-OG-Hil-PF11765-1-414.nuc"
cml.tree = "../p1-414.nwk"
cml.out_file = "mlc"

In [90]:
cml.set_options(
    noisy = 2,
    verbose = 0,
    seqtype = 1,
    CodonFreq = 2,
    clock = 0,
    model = 1,
    NSsites = [0],
    icode = 8,
    fix_kappa = 0,
    kappa = 2,
    fix_omega = 0,
    omega = .4,
    cleandata = 1,
    fix_blength = 0
)

In [91]:
cml.ctl_file = "codeml.ctl"
cml.write_ctl_file()

In [92]:
res = cml.run()

In [93]:
sum_res = {"branch-freer-cf2": res}

In [ ]:
for model, res in sum_res.items():
    